In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### EDA

In [ ]:
df = pd.read_csv("/kaggle/input/books-data-from-an-ecom-site/books_from_ecom.csv")
df.head()


In [ ]:
# If you want to See all the coumns just take the Transpose of the DataFrame
df.head(10).T

In [ ]:
# Let's Check the Shape of the DataFrame
df.shape

Data Set has 934 Rows and 11 Columns

In [ ]:
#Checks number of Unique enteries in the dataframe
df.nunique()

There is one Identifier column sr_no that needs to be removed

In [ ]:
df.dtypes

Mostly datatypes of the columns in DataFrame are object

In [ ]:
#Checking Is there any Null enteries present in the DataFrame

df.isna().sum()

In [ ]:
df.skew()

In [ ]:
df.describe()

### Univariate Analysis

In [ ]:
cat=[i for i in df.columns if df[i].nunique()<11]
cont=[i for i in df.columns if df[i].nunique()>=11 and i!='sr_no']

print(cat, cont)

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)

df.product_real_price.value_counts().plot.pie(autopct='%1.1f%%')
centre=plt.Circle((0,0),0.7,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre)
plt.subplot(1,2,2)
sns.countplot(x='product_real_price',data=df)
df['product_real_price'].value_counts()

Here we simply got the idea that the mostly books has 175 Price.

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
df['type'].value_counts().plot.pie(autopct='%1.1f%%')
centre=plt.Circle((0,0),0.7,fc='white')
fig=plt.gcf()
fig.gca().add_artist(centre)
plt.subplot(1,2,2)
sns.countplot(x='type',data=df)
df['type'].value_counts()

Most of the types of the books sold was Paperback type

In [ ]:
plt.figure(figsize=(25,8))
plt.subplot(1,2,1)
df['language'].value_counts().plot.pie(autopct='%1.1f%%')
centre=plt.Circle((0,0),0.7,fc='white')
fig=plt.gcf()
fig.gca().add_artist(centre)
plt.subplot(1,2,2)
sns.countplot(x='language',data=df)
df['language'].value_counts()

Most of the Books sold were in English language and then Hindi.

In [ ]:
df['product_real_price'].describe()

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(x='language',data=df)
df['language'].value_counts()

In [ ]:
fig , ax = plt.subplots(figsize=(15,10))
ax = sns.heatmap(df.corr(),
           annot=True,
            linewidth=0.5,
            fmt='.2f',
            cmap="YlGnBu",
           )

Here we can see a visible relation between the discount offer percent and the Product rating. Surely the people will give good rating if the discount is greater on a good product.

#### Feature Engineering

In [ ]:
df.head().T

In [ ]:
df.language.value_counts()

In [ ]:
df.nunique()


In [ ]:
#df_tst.product_real_price = df_tst[x.replace(",","") for x in 'product_real_price']:

        


In [ ]:

df_tst = df.copy()
df_tst['product_real_price'] = df_tst['product_real_price'].str.replace(',', '').astype(int)
df_tst['product_disc_price'] = df_tst['product_disc_price'].str.replace(',', '').astype(int)


df_tst.drop('sr_no',inplace=True,axis=1)
df_tst.drop('product_name',inplace=True,axis=1)
df_tst['on_promotion']=df_tst['on_promotion'].replace(to_replace=['yes','No'],value=[1,0])

#df_tst['product_disc_price']=df_tst['product_disc_price'].astype(int) 
df_tst=pd.get_dummies(df_tst)
        



### MODEL SELETION FOR PRICE PREDICTION

In [ ]:
from sklearn.model_selection import train_test_split
X = df_tst.drop('product_real_price',axis=1)
y = df_tst['product_real_price']
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.1,random_state=1)

In [ ]:
X_train.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1,random_state=42)
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)